In [1]:
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import compress_entropy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(compress_entropy.Compress().to(device), (3, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 768, 512]             216
               ELU-2          [-1, 8, 768, 512]               0
            Conv2d-3          [-1, 8, 768, 512]             576
               ELU-4          [-1, 8, 768, 512]               0
            Conv2d-5          [-1, 8, 768, 512]             576
               ELU-6          [-1, 8, 768, 512]               0
            Conv2d-7          [-1, 8, 768, 512]              64
         MaxPool2d-8          [-1, 8, 768, 512]               0
               ELU-9          [-1, 8, 768, 512]               0
           Conv2d-10         [-1, 16, 768, 512]              48
             Stem-11         [-1, 16, 768, 512]               0
           Conv2d-12          [-1, 8, 768, 512]             128
              ELU-13          [-1, 8, 768, 512]               0
           Conv2d-14          [-1, 8, 7

In [2]:
import shutil
import torch
import torch.nn as nn
import numpy as np
torch.manual_seed(100)
np.random.seed(100)
import compress_entropy
import generateCompressionImages

global counter 
counter = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def save_ckp(state, is_best, checkpoint_dir="./entropy_models/rest/", best_model_dir="./entropy_models/best/"):
    global counter 
    f_path = checkpoint_dir + str(counter) + '_checkpoint.pt'
    counter = counter + 1
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)


In [3]:
from torch.cuda.amp import autocast
loss_fn = nn.L1Loss(reduction='mean')  
startup = True
min_lr = 0.003
max_lr = 0.01
decay = 0.9
steps = 50  # might want to bump this up if the HPC is much faster than my computer
falling = True
start_epoch = 0
start_index = 0
momentum = 0.94
step_size = (max_lr-min_lr)/steps

printing = True
epochs = 100
batch_size = 4

model = compress_entropy.Compress().to(device).to(memory_format=torch.channels_last)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=momentum)


torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.autograd.set_detect_anomaly(False, check_nan=False)
torch.autograd.profiler.profile(enabled=False)
torch.autograd.profiler.emit_nvtx(enabled=False)
torch.backends.cudnn.benchmark = True
torch.set_grad_enabled(True)

In [4]:
torch.set_grad_enabled(True)
startup = True
for epoch in range(start_epoch, epochs):

    if startup:
        training = generateCompressionImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, startup = True)
        training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)
        min_lr *= batch_size**0.5  
        max_lr *= batch_size**0.5
        step_size = (max_lr-min_lr)/steps
        optimizer.param_groups[-1]['lr'] = max_lr
        los = [0]*10
    else:
        training = generateCompressionImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, epoch=epoch, startup = False)
        training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)


    for index, data in enumerate(training_loader):
        inputs, labels = data
        labels.to(memory_format=torch.channels_last)
        inputs.to(memory_format=torch.channels_last)

        with autocast():
            outputs = model(inputs)            
            loss = loss_fn(outputs, labels)

        loss.backward()
        if startup:
            los[index%10] = loss.item()
             
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
        else:
            los[index%10] = loss.item()  # should be deleted
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        if falling:
            optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
            if optimizer.param_groups[-1]['lr'] < min_lr:
                falling = False
                max_lr *= decay
                min_lr *= decay
                steps /= decay
                step_size = (max_lr-min_lr)/steps
                if printing:
                    print("Saving model !!")
                    print("Min lr")
                    print(min_lr)
                    print("Max lr")
                    print(max_lr)
                checkpoint = {'epoch': epoch, 'index': index, 'min_lr': min_lr, 'max_lr': max_lr, 'steps': steps, 'step_size': step_size, 'falling': falling, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'startup': startup}
                save_ckp(checkpoint, True)
                
                if startup and sum(los)/10 < 0.25 and max(los) < 0.35:
                    startup = False
                    max_lr /= 4
                    min_lr /= 4
                    steps *= 2
                    step_size = (max_lr-min_lr)/steps
                    falling = True
                    optimizer.param_groups[-1]['lr'] = max_lr
                    print("startup done ! !")
                    break
        else: 
            optimizer.param_groups[-1]['lr'] += step_size
            if optimizer.param_groups[-1]['lr'] > max_lr:
                falling = True

        if (startup and printing and index % 10  == 0):
            print("loss")
            print(los)
            print("lr")
            print(optimizer.param_groups[-1]['lr'])
        elif (not startup and printing and index % 100 == 0):
            print("loss")
            print(los)#loss.item())
            print("lr")
            print(optimizer.param_groups[-1]['lr'])

loss
[0.026794178411364555, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lr
0.01972
loss
[0.5383299589157104, 0.023624129593372345, 0.5478426814079285, 0.5295305252075195, 0.6313802599906921, 0.5789573788642883, 0.4879108965396881, 0.5231168866157532, 0.5607575178146362, 0.42612409591674805]
lr
0.01692000000000001
loss
[0.33692383766174316, 0.5252764225006104, 0.3614864945411682, 0.3523622751235962, 0.4052997827529907, 0.38854989409446716, 0.3138146698474884, 0.4220317304134369, 0.32482099533081055, 0.3618474006652832]
lr
0.014120000000000011
loss
[0.23011398315429688, 0.46142420172691345, 0.2910204529762268, 0.25948280096054077, 0.26048290729522705, 0.31726810336112976, 0.32479944825172424, 0.29252955317497253, 0.32818686962127686, 0.34267881512641907]
lr
0.011320000000000004
loss
[0.32769277691841125, 0.22290413081645966, 0.38566309213638306, 0.2618319094181061, 0.27776435017585754, 0.2832016050815582, 0.2530708312988281, 0.2946242392063141, 0.2073320597410202, 0.36012473702430725]
lr
0.00851999999999